In [7]:
import json
from argparse import Namespace
from pathlib import Path

from typing import Dict, List, Tuple, Set, Union

In [2]:
PROJECT_DIR =  Path('..').absolute()
PROJECT_DIR

PosixPath('/Users/ziyang/Projects/aca/nyu-nlp-homework-3/src/..')

In [3]:
args = Namespace(
    train_file_path=PROJECT_DIR / 'data' / 'WSJ_02-21.pos',
    test_file_path=PROJECT_DIR / 'data' / 'WSJ_24.words',
)

In [4]:
lines = [line.strip() for line in open(args.train_file_path, 'r')]

In [5]:
lines = [
    'the\tDT',
    'cat\tNN',
    'sat\tVBD',
    'on\tIN',
    'the\tDT',
    'mat\tNN',
    '.\t.',
    '',
]

In [6]:
word_count: Dict[str, int] = {}
word_tag_count: Dict[Tuple[str, str], int] = {}
tag_count: Dict[str, int] = {}
tag_tag_count: Dict[Tuple[str, str], int] = {}

last_tag = 'B'
for line in lines:
    if last_tag == 'B':
        tag_count['B'] = tag_count.get('B', 0) + 1
    
    line = line.strip()
    if line:
        word, tag = [x.strip() for x in line.split("\t")]
        word = word.lower()
    else:
        word = ''
        tag = 'E'
    
    tag_count[tag] = tag_count.get(tag, 0) + 1
    tag_tag_count[(last_tag, tag)] = tag_tag_count.get((last_tag, tag), 0) + 1
    if word:
        word_count[word] = word_count.get(word, 0) + 1
        word_tag_count[(word, tag)] = word_tag_count.get((word, tag), 0) + 1

    if tag != 'E':
        last_tag = tag
    else:
        last_tag = 'B'


In [10]:
# clean for unknown word
affixes: List[Union[List[str], str]] = [['able', 'ible'], 'al', 'an', 'ar', 'ed', 'en', 'er', 'est', 'ful', 'ic', 'ing', 'ish', 'ive', 'less', 'ly', 'ment', 'ness', 'or', 'ous', 'y']

new_word_count = word_count.copy()

for word, count in word_count.items():
    if count > 1:
        continue
    word_class = "UNKNOWN"
    for affix in affixes:
        if type(affix) == list:
            word_affix_class = affix[0].upper()
            selected = False
            for affix_item in affix:
                if word.endswith(affix_item):
                    word_class = f"UNKNOWN_AFFIXED_WITH_{word_affix_class}"
                    selected = True
                    break
            if selected:
                break
        else:
            if word.endswith(affix):
                word_class = f"UNKNOWN_AFFIXED_WITH_{affix.upper()}"
                break
    new_word_count.pop(word)
    new_word_count[word_class] = new_word_count.get(word_class, 0) + 1
    for tag in tag_count.keys():
        original_word_tag_count = word_tag_count.pop((word, tag), 0)
        word_tag_count[(word_class, tag)] = word_tag_count.get((word_class, tag), 0) + original_word_tag_count


In [41]:
trans_prob: Dict[Tuple[str, str], float] = {} # (tag1, tag2) -> prob
for tag1 in tag_count.keys():
    for tag2 in tag_count.keys():
        total_tag1 = tag_count.get(tag1, 0)
        tag1_followed_by_tag2 = tag_tag_count.get((tag1, tag2), 0)
        if total_tag1 == 0:
            prob = 0
        else:
            prob = tag1_followed_by_tag2 / total_tag1
        trans_prob[(tag1, tag2)] = prob

In [ ]:
trans_prob

In [46]:
emission_prob: Dict[Tuple[str, str], float] = {} # (word, tag) -> prob
for word in word_set:
    for tag in tag_count.keys():
        word_tag = word_tag_count.get((word, tag), 0)
        tag_total = tag_count.get(tag, 0)
        if tag_total == 0:
            prob = 0
        else:
            prob = word_tag / tag_total
        emission_prob[(word, tag)] = prob

In [47]:
emission_prob

{('sat', 'B'): 0.0,
 ('sat', 'DT'): 0.0,
 ('sat', 'NN'): 0.0,
 ('sat', 'VBD'): 1.0,
 ('sat', 'IN'): 0.0,
 ('sat', '.'): 0.0,
 ('sat', 'E'): 0.0,
 ('mat', 'B'): 0.0,
 ('mat', 'DT'): 0.0,
 ('mat', 'NN'): 0.5,
 ('mat', 'VBD'): 0.0,
 ('mat', 'IN'): 0.0,
 ('mat', '.'): 0.0,
 ('mat', 'E'): 0.0,
 ('cat', 'B'): 0.0,
 ('cat', 'DT'): 0.0,
 ('cat', 'NN'): 0.5,
 ('cat', 'VBD'): 0.0,
 ('cat', 'IN'): 0.0,
 ('cat', '.'): 0.0,
 ('cat', 'E'): 0.0,
 ('.', 'B'): 0.0,
 ('.', 'DT'): 0.0,
 ('.', 'NN'): 0.0,
 ('.', 'VBD'): 0.0,
 ('.', 'IN'): 0.0,
 ('.', '.'): 1.0,
 ('.', 'E'): 0.0,
 ('the', 'B'): 0.0,
 ('the', 'DT'): 1.0,
 ('the', 'NN'): 0.0,
 ('the', 'VBD'): 0.0,
 ('the', 'IN'): 0.0,
 ('the', '.'): 0.0,
 ('the', 'E'): 0.0,
 ('on', 'B'): 0.0,
 ('on', 'DT'): 0.0,
 ('on', 'NN'): 0.0,
 ('on', 'VBD'): 0.0,
 ('on', 'IN'): 1.0,
 ('on', '.'): 0.0,
 ('on', 'E'): 0.0}

In [50]:
lines = [line.strip() for line in open(args.test_file_path, 'r')]